<a href="https://colab.research.google.com/github/victory256/Sample-ML-Repo/blob/hw11/%D0%AF%D0%BA%D1%83%D0%B1%D0%BE%D0%B2%D0%B0_hw11_text_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем оценку на основе отзыва

1. <a href="#look_to_dataset">Посмотрим на имеющийся datasеt</a>
2. <a href="#choose_features">Выбираем признаки</a>
3. <a href="#fit_predict">Обучаем модель и предсказываем класс</a>
4. <a href="#look_features">Внимательно смотрим признаки</a>
5. <a href="#submit">Сохраняем submit</a>

<a name="look_to_dataset"></a>
# 1. Посмотрим на имеющийся datasеt

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import Counter
import copy
import re
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import wordnet, pos_tag, WordNetLemmatizer, ngrams 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
sw_eng = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data_train = pd.read_csv('train.csv', encoding= 'unicode_escape')
data_test = pd.read_csv('test.csv', encoding= 'unicode_escape')
SentiWordNet = pd.read_csv('SentiWordNet.txt', sep='\t')

f = open('positive-words.txt', 'r')   
positive_words = f.read()
positive_words = positive_words.split("\n")
f.close()
f = open('negative-words.txt', 'r', encoding= 'unicode_escape')   
negative_words = f.read()
negative_words = negative_words.split("\n")
f.close()

In [4]:
data_train.head(7)

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0
5,5,Everest,"Good atmosphere, food and drinks not available","I reached the hotel by car, felt good for co-o...",71.0
6,6,Metro Grand,Lovely hotel,The hotel is pretty clean with excellent beddi...,80.0


In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            2351 non-null   int64  
 1   Hotel_name    2351 non-null   object 
 2   Review_Title  2136 non-null   object 
 3   Review_Text   2351 non-null   object 
 4   Rating        2351 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 92.0+ KB


In [6]:
data_train.isna().sum()

Id                0
Hotel_name        0
Review_Title    215
Review_Text       0
Rating            0
dtype: int64

Переведем в нижний регистр, уберем все знаки препинания и цифры, уберем стоп-слова в Review_Title и Review_Text.

In [7]:
def text_change(text_str):   
  text = re.sub(r'[^a-z]',' ',str(text_str).lower())   # переводим в нижний регистр и убираем все кроме букв
  text = re.sub(r'\s+[a-zA-Z]\s+',' ',text)   # заменяем все одиночные символы, имеющие пробелы с обеих сторон, одним пробелом
  text = re.sub(r'\s+',' ',text)   # заменяем один или несколько пробелов на один
  clean_text = ' '.join([word for word in text.split() if not word in sw_eng])  # убираем слова из стоп-листа
  return clean_text

In [8]:
data_train_1 = data_train.copy(deep=True)

data_train_1['Title'] = data_train_1['Review_Title'].apply(text_change)
data_train_1['Text'] = data_train_1['Review_Text'].apply(text_change)

In [9]:
data_train_1

,Id,Hotel_name,Review_Title,Review_Text,Rating,Title,Text
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,refuge chennai,excellent room exercise facility around atmosp...
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,hilton chennai,comfortable felt safe staff helpful respectful...
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rating shown websites pricing ok,worth rating shown service good room well main...
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,good stay,first nice courteous staff one con stay time c...
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,needs improvement,overall ambience hotel good room facilities ne...
...,...,...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,nan,impressive service staff areas good restaurant...
2347,2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,homely villa,new woodlands chennai gave homely experience l...
2348,2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,nice accommodation facilities,awesome liked neatness maintenance facilities ...
2349,2349,The Residency Chennai,The Residency Good Centrally located Hotel,The overall experience was good. However the w...,80.0,residency good centrally located hotel,overall experience good however wi fi getting ...


Сделаем Лемматизацию текста, чтобы привести слова к начальной форме.

In [10]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

def my_lemmatizer(sent):
  if sent!='nan':
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])  
  else:
    return '' 

In [11]:
data_train_1['Title_lemmatizer'] = data_train_1['Title'].apply(my_lemmatizer)
data_train_1['Text_lemmatizer'] = data_train_1['Text'].apply(my_lemmatizer)

data_train_1['Title_Text_lemmatizer'] = data_train_1['Title_lemmatizer']+' '+data_train_1['Text_lemmatizer']

Добавим количество positive_words и negative_words в Review_Title и Review_Text.

In [12]:
def positive_count(text_str):   # показываем положительные слова в тексте
  return [word for word in text_str.split() if word in positive_words]

def negative_count(text_str):   # показываем отрицательные слова в тексте
  return [word for word in text_str.split() if word in negative_words]

def text_count(text_str):   # считаем количество слов в тексте
  return len(text_str)

In [13]:
data_train_1['Title_positive_text'] = data_train_1['Title_lemmatizer'].apply(positive_count)
data_train_1['Text_positive_text'] = data_train_1['Text_lemmatizer'].apply(positive_count)
data_train_1['Title_positive_count'] = data_train_1['Title_positive_text'].apply(text_count)
data_train_1['Text_positive_count'] = data_train_1['Text_positive_text'].apply(text_count)

data_train_1['Title_negative_text'] = data_train_1['Title_lemmatizer'].apply(negative_count)
data_train_1['Text_negative_text'] = data_train_1['Text_lemmatizer'].apply(negative_count)
data_train_1['Title_negative_count'] = data_train_1['Title_negative_text'].apply(text_count)
data_train_1['Text_negative_count'] = data_train_1['Text_negative_text'].apply(text_count)

data_train_1['Negative_count'] = data_train_1['Title_negative_count']+data_train_1['Text_negative_count']
data_train_1['Positive_count'] = data_train_1['Title_positive_count']+data_train_1['Text_positive_count']

Посмотрим есть ли в текстах повторяющиеся N-граммы.

In [14]:
title_str = ' '.join(data_train_1['Title_lemmatizer']).split()
q = list(ngrams(title_str,n=2))
counts = Counter(q)
counts.most_common()[:100]

[(('good', 'hotel'), 140),
 (('stay', 'good'), 88),
 (('value', 'money'), 82),
 (('hotel', 'good'), 78),
 (('nice', 'hotel'), 77),
 (('good', 'stay'), 73),
 (('hotel', 'stay'), 70),
 (('place', 'stay'), 59),
 (('good', 'location'), 54),
 (('good', 'service'), 37),
 (('conversation', 'much'), 33),
 (('nice', 'stay'), 32),
 (('experience', 'good'), 31),
 (('money', 'good'), 30),
 (('stay', 'nice'), 30),
 (('service', 'good'), 29),
 (('good', 'experience'), 29),
 (('location', 'good'), 29),
 (('hotel', 'nice'), 28),
 (('great', 'hotel'), 26),
 (('good', 'place'), 26),
 (('pleasant', 'stay'), 25),
 (('good', 'food'), 25),
 (('nice', 'place'), 23),
 (('hotel', 'great'), 22),
 (('budget', 'hotel'), 22),
 (('best', 'hotel'), 22),
 (('comfortable', 'stay'), 22),
 (('worth', 'money'), 21),
 (('food', 'good'), 20),
 (('room', 'service'), 20),
 (('excellent', 'stay'), 20),
 (('helpful', 'staff'), 20),
 (('bad', 'experience'), 20),
 (('room', 'good'), 20),
 (('hotel', 'review'), 19),
 (('average',

In [15]:
title_str = ' '.join(data_train_1['Text_lemmatizer']).split()
q = list(ngrams(title_str,n=2))
counts = Counter(q)
counts.most_common()[:100]

[(('hotel', 'good'), 225),
 (('good', 'hotel'), 221),
 (('room', 'service'), 180),
 (('value', 'money'), 152),
 (('service', 'good'), 139),
 (('room', 'clean'), 126),
 (('stay', 'hotel'), 117),
 (('hotel', 'staff'), 110),
 (('good', 'location'), 105),
 (('good', 'room'), 101),
 (('food', 'good'), 100),
 (('hotel', 'room'), 93),
 (('room', 'good'), 92),
 (('good', 'service'), 92),
 (('location', 'good'), 91),
 (('good', 'food'), 89),
 (('also', 'good'), 83),
 (('staff', 'good'), 82),
 (('location', 'hotel'), 75),
 (('clean', 'room'), 71),
 (('hotel', 'locate'), 71),
 (('staff', 'friendly'), 71),
 (('place', 'stay'), 68),
 (('hotel', 'stay'), 67),
 (('neat', 'clean'), 63),
 (('well', 'maintain'), 62),
 (('breakfast', 'good'), 61),
 (('stay', 'good'), 60),
 (('overall', 'good'), 60),
 (('hotel', 'nice'), 57),
 (('complimentary', 'breakfast'), 56),
 (('nice', 'hotel'), 55),
 (('good', 'stay'), 54),
 (('staff', 'helpful'), 53),
 (('good', 'experience'), 53),
 (('hotel', 'location'), 52),
 (

В основном словосочетания позитивные. Попробуем их использовать при векторизации, будем смотреть на N-граммы с частотой больше 20. 

<a name="choose_features"></a>
# 2. Выберем признаки для обучения

Разделим на train и test исходный train-dataset.

In [16]:
data_train_2 = data_train_1.drop(columns=['Review_Title', 'Review_Text'])
X_train, X_test, y_train, y_test = train_test_split(data_train_2, data_train_2['Rating'], test_size=0.2, random_state=42)

Переведем в вектора текст в столбцах Title_lemmatizer и Text_lemmatizer

In [17]:
# посмотрим метрику только на столбце Title
count_vect_title = CountVectorizer(max_df=0.7)
X_train_counts_title = count_vect_title.fit_transform(X_train['Title_lemmatizer'])
X_test_counts_title = count_vect_title.transform(X_test['Title_lemmatizer'])

svm_title = SGDClassifier()
svm_title.fit(X_train_counts_title, y_train)
predicted = svm_title.predict(X_test_counts_title)
print('Title')
print(metrics.classification_report(y_test, predicted))

# посмотрим метрику только на столбце Text
count_vect_text = CountVectorizer(max_df=0.7)
X_train_counts_text = count_vect_text.fit_transform(X_train['Text_lemmatizer'])
X_test_counts_text = count_vect_text.transform(X_test['Text_lemmatizer'])

svm_text = SGDClassifier()
svm_text.fit(X_train_counts_text, y_train)
predicted = svm_text.predict(X_test_counts_text)
print('Text')
print(metrics.classification_report(y_test, predicted))

Title
              precision    recall  f1-score   support

        20.0       0.33      0.29      0.31        17
        29.0       0.18      0.17      0.17        12
        40.0       0.80      0.17      0.28        24
        43.0       0.50      0.10      0.16        21
        57.0       0.30      0.21      0.25        28
        60.0       0.15      0.45      0.23        31
        65.0       0.23      0.10      0.14        29
        71.0       0.33      0.30      0.31        63
        80.0       0.38      0.36      0.37        61
        86.0       0.46      0.62      0.53       102
       100.0       0.55      0.39      0.45        83

    accuracy                           0.37       471
   macro avg       0.38      0.29      0.29       471
weighted avg       0.41      0.37      0.36       471

Text
              precision    recall  f1-score   support

        20.0       0.36      0.29      0.32        17
        29.0       0.17      0.17      0.17        12
        40.0 

In [18]:
Counter(data_train_2['Rating'])

Counter({20.0: 88,
         29.0: 69,
         40.0: 88,
         43.0: 87,
         57.0: 144,
         60.0: 172,
         65.0: 163,
         71.0: 327,
         80.0: 307,
         86.0: 465,
         100.0: 441})

Видим, что на признаке  Text_lemmatizer все метрики получаются в целом выше. Тогда попробуем объединить Title_lemmatizer и Text_lemmatizer

In [19]:
# посмотрим метрику только на столбце Title и Text
count_vect_title_text = CountVectorizer(max_df=0.7)
X_train_counts_title_text = count_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_counts_title_text = count_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

svm_title_text = SGDClassifier()
svm_title_text.fit(X_train_counts_title_text, y_train)
predicted = svm_title_text.predict(X_test_counts_title_text)
print('Title и Text')
print(metrics.classification_report(y_test, predicted))

Title и Text
              precision    recall  f1-score   support

        20.0       0.39      0.41      0.40        17
        29.0       0.29      0.17      0.21        12
        40.0       0.64      0.38      0.47        24
        43.0       0.45      0.24      0.31        21
        57.0       0.30      0.29      0.29        28
        60.0       0.41      0.42      0.41        31
        65.0       0.29      0.34      0.32        29
        71.0       0.67      0.54      0.60        63
        80.0       0.62      0.66      0.63        61
        86.0       0.65      0.66      0.65       102
       100.0       0.59      0.77      0.67        83

    accuracy                           0.55       471
   macro avg       0.48      0.44      0.45       471
weighted avg       0.55      0.55      0.54       471



Оставим объединенный текстовый признак, кажется что для него метрики чуть выше, чем только для Text столбца.

Добавим к векторизированным столбцам признаки, которые получили при подсчете положительных и отрицательных слов. Прежде чем добавлять новые столбцы к векторизации CountVectorizer переведем их в DF. И проверим, что результат обучения не меняется.

In [20]:
counts_train = pd.DataFrame(X_train_counts_title_text.toarray(), index=X_train.Id, columns=count_vect_title_text.get_feature_names())
counts_test = pd.DataFrame(X_test_counts_title_text.toarray(), index=X_test.Id, columns=count_vect_title_text.get_feature_names())
svm = SGDClassifier()
svm.fit(counts_train, y_train)
predicted = svm.predict(counts_test)
print(metrics.classification_report(y_test, predicted))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


              precision    recall  f1-score   support

        20.0       0.29      0.29      0.29        17
        29.0       0.50      0.25      0.33        12
        40.0       0.53      0.33      0.41        24
        43.0       0.55      0.29      0.37        21
        57.0       0.35      0.39      0.37        28
        60.0       0.48      0.39      0.43        31
        65.0       0.47      0.31      0.38        29
        71.0       0.51      0.56      0.53        63
        80.0       0.58      0.70      0.64        61
        86.0       0.59      0.63      0.61       102
       100.0       0.62      0.72      0.67        83

    accuracy                           0.54       471
   macro avg       0.50      0.44      0.46       471
weighted avg       0.54      0.54      0.53       471



Но почему-то получаем немного другой результат после перевода векторизации в toarray(). Не знаю в чем причина, но думаю, что так быть не должно. 

In [21]:
X_train_title_text = pd.merge(counts_train, X_train[['Negative_count','Positive_count']], left_on='Id', right_index=True)
X_test_title_text = pd.merge(counts_test, X_test[['Negative_count','Positive_count']], left_on='Id', right_index=True)
svm = SGDClassifier(random_state=42)
svm.fit(X_train_title_text, y_train)
predicted = svm.predict(X_test_title_text)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

        20.0       0.38      0.35      0.36        17
        29.0       1.00      0.25      0.40        12
        40.0       0.71      0.42      0.53        24
        43.0       0.39      0.33      0.36        21
        57.0       0.00      0.00      0.00        28
        60.0       0.32      0.58      0.41        31
        65.0       0.56      0.17      0.26        29
        71.0       0.46      0.59      0.51        63
        80.0       0.73      0.59      0.65        61
        86.0       0.60      0.65      0.62       102
       100.0       0.56      0.76      0.65        83

    accuracy                           0.53       471
   macro avg       0.52      0.43      0.43       471
weighted avg       0.53      0.53      0.51       471



Добавим два признака Negative_count и Positive_count. Судя по f1-мере  улучшение есть. 

Не будем добавлять наши признаки, которые получили на основе текста. Будем работать только с результатом векторизации. Потому что думаю, что результат не должен меняться при переходе от матрицы к DF. А у меня меняется и исправить это не получилось - не вижу в чем причина. 

<a name="fit_predict"></a>
# 3. Обучаем модель и предсказываем класс

Посмотрим на результат для разной векторизации для одного алгоритма SGDClassifier

In [22]:
# посмотрим метрику только на столбце Title и Text
count_vect_title_text = CountVectorizer(max_df=0.7, ngram_range=(1,2))
X_train_counts_title_text = count_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_counts_title_text = count_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

svm_title_text = SGDClassifier()
svm_title_text.fit(X_train_counts_title_text, y_train)
predicted = svm_title_text.predict(X_test_counts_title_text)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

        20.0       0.43      0.53      0.47        17
        29.0       0.29      0.17      0.21        12
        40.0       0.64      0.38      0.47        24
        43.0       0.50      0.24      0.32        21
        57.0       0.35      0.32      0.33        28
        60.0       0.42      0.42      0.42        31
        65.0       0.50      0.24      0.33        29
        71.0       0.51      0.54      0.52        63
        80.0       0.70      0.64      0.67        61
        86.0       0.60      0.66      0.63       102
       100.0       0.58      0.80      0.67        83

    accuracy                           0.55       471
   macro avg       0.50      0.45      0.46       471
weighted avg       0.55      0.55      0.54       471



In [23]:
tfidf_vect_title_text = TfidfVectorizer(max_df=0.7, ngram_range=(1,2))
X_train_tfidf_title_text = tfidf_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_tfidf_title_text = tfidf_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

svm_title_text = SGDClassifier()
svm_title_text.fit(X_train_tfidf_title_text, y_train)
predicted = svm_title_text.predict(X_test_tfidf_title_text)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

        20.0       0.44      0.47      0.46        17
        29.0       0.38      0.25      0.30        12
        40.0       0.80      0.33      0.47        24
        43.0       0.75      0.29      0.41        21
        57.0       0.38      0.32      0.35        28
        60.0       0.52      0.39      0.44        31
        65.0       0.58      0.24      0.34        29
        71.0       0.43      0.54      0.48        63
        80.0       0.70      0.57      0.63        61
        86.0       0.51      0.70      0.59       102
       100.0       0.66      0.80      0.72        83

    accuracy                           0.55       471
   macro avg       0.56      0.44      0.47       471
weighted avg       0.57      0.55      0.54       471



Векторизация TfidfVectorizer и CountVectorizer показали почти одинаковые метрики после применения SGDClassifier. Но в целом метрики по классам чуть выше для TfidfVectorizer. Наверное лучше применять нормализованное представление tf-idf.

Сравним метрики для векторизации CountVectorizer и для векторизации CountVectorizer + нормализация TfidfTransformer.

In [24]:
count_vect_title_text = CountVectorizer(max_df=0.7, ngram_range=(1,5))
X_train_counts_title_text = count_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_counts_title_text = count_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

svm_title_text = SGDClassifier()
svm_title_text.fit(X_train_counts_title_text,y_train)
print(metrics.classification_report(y_test, svm_title_text.predict(X_test_counts_title_text)))

              precision    recall  f1-score   support

        20.0       0.42      0.47      0.44        17
        29.0       0.50      0.17      0.25        12
        40.0       0.44      0.46      0.45        24
        43.0       0.57      0.19      0.29        21
        57.0       0.40      0.29      0.33        28
        60.0       0.44      0.48      0.46        31
        65.0       0.41      0.31      0.35        29
        71.0       0.60      0.48      0.53        63
        80.0       0.46      0.62      0.53        61
        86.0       0.65      0.52      0.58       102
       100.0       0.55      0.83      0.66        83

    accuracy                           0.52       471
   macro avg       0.49      0.44      0.44       471
weighted avg       0.53      0.52      0.51       471



In [25]:
count_vect_title_text = CountVectorizer(max_df=0.7, ngram_range=(1,5))
X_train_counts_title_text = count_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_counts_title_text = count_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

from sklearn.feature_extraction.text import TfidfTransformer
# преобразуем матрицу подсчета в нормализованное представление tf-idf
tfidf_transf = TfidfTransformer()
X_train_tfidf = tfidf_transf.fit_transform(X_train_counts_title_text)
X_test_tfidf = tfidf_transf.transform(X_test_counts_title_text)

svm_title_text = SGDClassifier()
svm_title_text.fit(X_train_tfidf,y_train)
print(metrics.classification_report(y_test, svm_title_text.predict(X_test_tfidf)))

              precision    recall  f1-score   support

        20.0       0.54      0.41      0.47        17
        29.0       0.50      0.17      0.25        12
        40.0       1.00      0.33      0.50        24
        43.0       1.00      0.19      0.32        21
        57.0       0.44      0.29      0.35        28
        60.0       0.73      0.35      0.48        31
        65.0       0.88      0.24      0.38        29
        71.0       0.41      0.54      0.47        63
        80.0       0.79      0.56      0.65        61
        86.0       0.47      0.85      0.60       102
       100.0       0.72      0.77      0.74        83

    accuracy                           0.56       471
   macro avg       0.68      0.43      0.47       471
weighted avg       0.64      0.56      0.55       471



Нормализованны вектора tf-idf показали более высокие метрики по классам, поэтому выберем его. Теперь посмотрим разные модели классификации

Сравним метрики для векторизации CountVectorizer и для векторизации CountVectorizer + нормализация TfidfTransformer.

Теперь попробуем LinearSVC

In [26]:
# сразу преобразуем набор признаков в нормализованное представление tf-idf.
tfidf_vect_title_text = TfidfVectorizer(max_df=0.7, ngram_range=(1,5))
X_train_tfidf_title_text = tfidf_vect_title_text.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_tfidf_title_text = tfidf_vect_title_text.transform(X_test['Title_Text_lemmatizer'])

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df_est = pd.DataFrame()
list_est = [SGDClassifier(),LinearSVC(),LogisticRegression(),RandomForestClassifier(),
            GradientBoostingClassifier(),DecisionTreeClassifier(),SVC(),KNeighborsClassifier(n_neighbors=11)]
for i in list_est:
  i.fit(X_train_tfidf_title_text,y_train)
  y_pred = i.predict(X_test_tfidf_title_text)
  df_est = df_est.append({'method':i, 'accuracy':accuracy_score(y_test,y_pred),
                          'precision':precision_score(y_test,y_pred, average='macro'),
                          'recall':recall_score(y_test,y_pred, average='macro'),
                          'f1':f1_score(y_test,y_pred, average='macro')}, 
                          ignore_index=True)
df_est

,method,accuracy,precision,recall,f1
0,SGDClassifier(),0.579618,0.674350,0.448739,0.496115
1,LinearSVC(),0.579618,0.669585,0.447597,0.492393
2,LogisticRegression(),0.524416,0.804675,0.339029,0.372935
3,"(DecisionTreeClassifier(max_features='auto', r...",0.513800,0.675087,0.382426,0.411315
4,([DecisionTreeRegressor(criterion='friedman_ms...,0.490446,0.437329,0.401418,0.394248
5,DecisionTreeClassifier(),0.507431,0.455519,0.418535,0.425757
6,SVC(),0.526539,0.921606,0.368060,0.442924
7,KNeighborsClassifier(n_neighbors=11),0.269639,0.255927,0.207576,0.200337


Видим, что все алгоритмы показывают низкую accurace около 0,5. Самый лучший результат 0,58 показал SGDClassifier и LinearSVC. Значит надо больше поработать с исходными данными.

<a name="look_features"></a>
# 4. Внимательно смотрим признаки

Выше мы пробовали переводить результат векторизации CountVectorizer в dataFrame и получили немного другие метрики после SGDClassifier, по сравнению с метриками SGDClassifier на основе CountVectorizer. Поэтому решили пока не добавлять новые признаки. Таблица выше показывает, что только векторизация TfidfVectorizer дает accuracy около 0,57-0,58 - это мало. Поэтому 


*   векторизируем Title и Text, и переведем в DataFrame
*   добавим/вычтем к значениям слова Positive/Negative коэффициент из файла SentiWordNet.txt
*   добавим два слолбца: количество положительных и отрицательных слов

и посмотрим как это повлиет потом на результат обучения модели.




In [27]:
data_train_1.head(5)

,Id,Hotel_name,Review_Title,Review_Text,Rating,Title,Text,Title_lemmatizer,Text_lemmatizer,Title_Text_lemmatizer,Title_positive_text,Text_positive_text,Title_positive_count,Text_positive_count,Title_negative_text,Text_negative_text,Title_negative_count,Text_negative_count,Negative_count,Positive_count
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,refuge chennai,excellent room exercise facility around atmosp...,refuge chennai,excellent room exercise facility around atmosp...,refuge chennai excellent room exercise facilit...,[],"[excellent, calm, comfortable, excellent]",0,4,[],[],0,0,0,4
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,hilton chennai,comfortable felt safe staff helpful respectful...,hilton chennai,comfortable felt safe staff helpful respectful...,hilton chennai comfortable felt safe staff hel...,[],"[comfortable, safe, helpful, respectful]",0,4,[],[],0,0,0,4
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rating shown websites pricing ok,worth rating shown service good room well main...,worth rating show website price ok,worth rating show service good room well maint...,worth rating show website price ok worth ratin...,[worth],"[worth, good, well, spacious, work, good, wise...",1,8,[],[poor],0,1,1,9
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,good stay,first nice courteous staff one con stay time c...,good stay,first nice courteous staff one con stay time c...,good stay first nice courteous staff one con s...,[good],"[nice, courteous, neat, exceptional]",1,4,[],"[inappropriate, disappointed]",0,2,2,5
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,needs improvement,overall ambience hotel good room facilities ne...,need improvement,overall ambience hotel good room facility need...,need improvement overall ambience hotel good r...,[improvement],"[good, improve]",1,2,[],[],0,0,0,3


Возьмем часть столбцов из таблицы, которую сделали в первом пункте.

In [28]:
data_train_3 = data_train_1[['Id', 'Hotel_name','Rating','Title_Text_lemmatizer','Negative_count','Positive_count']]
data_train_3

,Id,Hotel_name,Rating,Title_Text_lemmatizer,Negative_count,Positive_count
0,0,Park Hyatt,80.0,refuge chennai excellent room exercise facilit...,0,4
1,1,Hilton Chennai,100.0,hilton chennai comfortable felt safe staff hel...,0,4
2,2,The Royal Regency,71.0,worth rating show website price ok worth ratin...,1,9
3,3,Rivera,86.0,good stay first nice courteous staff one con s...,2,5
4,4,Park Hyatt,86.0,need improvement overall ambience hotel good r...,0,3
...,...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,80.0,impressive service staff area good restaurant...,0,9
2347,2347,New Woodlands,71.0,homely villa new woodland chennai give homely ...,0,6
2348,2348,Samudra Residency,100.0,nice accommodation facility awesome like neatn...,0,7
2349,2349,The Residency Chennai,80.0,residency good centrally locate hotel overall ...,1,2


В таблице SentiWordNet сгруппируем данные по Word, чтобы для каждого слова получить среднее значение Positive/Negative. Потом сделаем векторизацию TfidfVectorizer. И для каждой строки умножим значение получившегося признака-слова на его вес (Positive-Negative) из таблицы SentiWordNet_df.

In [29]:
some_value = 'good'
SentiWordNet_mean = SentiWordNet.groupby('Word',as_index=False).mean()
SentiWordNet_df = pd.DataFrame(SentiWordNet_mean)
SentiWordNet_df.loc[SentiWordNet_df['Word'] == some_value]

,Word,Positive,Negative
58634,good,0.569444,0.00463


In [30]:
X_train, X_test, y_train, y_test = train_test_split(data_train_3, data_train_3['Rating'], test_size=0.2, random_state=42)
# преобразуем набор признаков в нормализованное представление tf-idf
tfidf = TfidfVectorizer(max_df=0.7) #, ngram_range=(1,5))
X_train_tfidf = tfidf.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_tfidf = tfidf.transform(X_test['Title_Text_lemmatizer'])
# переводим результат векторизации в dataFrame
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.Id, columns=tfidf.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.Id, columns=tfidf.get_feature_names_out())

In [31]:
import time
from tqdm import tqdm
for n_gram in tqdm(tfidf.get_feature_names_out()):
  weight_word = 0
  #print(n_gram)
  for word in n_gram.split():
    if word in set(SentiWordNet_df['Word']):
      weight_word = weight_word + float(SentiWordNet_df.loc[SentiWordNet_df['Word'] == word]['Positive'])
      weight_word = weight_word - float(SentiWordNet_df.loc[SentiWordNet_df['Word'] == word]['Negative'])
      #print(word,'=',weight_word)
  X_train_tfidf_df[n_gram] = weight_word * X_train_tfidf_df[n_gram]   
  X_test_tfidf_df[n_gram] = weight_word * X_test_tfidf_df[n_gram] 

100%|██████████| 3527/3527 [04:21<00:00, 13.48it/s]


In [32]:
#from sklearn.preprocessing import StandardScaler
#pca = StandardScaler()
#X_train_tfidf_df=pca.fit_transform(X_train_tfidf_df)
#X_test_tfidf_df=pca.transform(X_test_tfidf_df) # применяем к test нормировку с mu и sigma из train

svm = SGDClassifier()
svm.fit(X_train_tfidf_df, y_train)
predicted = svm.predict(X_test_tfidf_df)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

        20.0       0.33      0.29      0.31        17
        29.0       0.31      0.42      0.36        12
        40.0       0.50      0.12      0.20        24
        43.0       0.00      0.00      0.00        21
        57.0       0.33      0.21      0.26        28
        60.0       0.50      0.19      0.28        31
        65.0       0.11      0.10      0.11        29
        71.0       0.57      0.13      0.21        63
        80.0       0.22      0.82      0.34        61
        86.0       0.56      0.37      0.45       102
       100.0       0.67      0.51      0.58        83

    accuracy                           0.35       471
   macro avg       0.37      0.29      0.28       471
weighted avg       0.45      0.35      0.34       471



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy стало еще ниже = 0,37, даже хуже, чем 0,57, которые получили только за счет векторизации. 

Попробуем добавить ngram_range=(1,2), но думаю, что это тоже не спасет ситуацию. Получилось = 0,47. Да, N-граммы повысили accuracy с 0,37 до 0,47, но это ниже, чем 0,57 (с ипользованием только векторизации). При добавлении N-грамм = (1,2) количество признаков увеличилось с 3_400 до 24_000 и время счета = с 6 минут до 1 часа. Не будем проверять для N-граммы = (1,3), думаю что accuracy до 0,7 это не поднимет, а считаться будет слишком долго.

In [56]:
# преобразуем набор признаков в нормализованное представление tf-idf
tfidf = TfidfVectorizer(max_df=0.7, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_tfidf = tfidf.transform(X_test['Title_Text_lemmatizer'])
# переводим результат векторизации в dataFrame
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.Id, columns=tfidf.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.Id, columns=tfidf.get_feature_names_out())

# умножаем на веса из SentiWordNet_df
for n_gram in tqdm(tfidf.get_feature_names_out()):
  weight_word = 0
  #print(n_gram)
  for word in n_gram.split():
    if word in set(SentiWordNet_df['Word']):
      weight_word = weight_word + float(SentiWordNet_df.loc[SentiWordNet_df['Word'] == word]['Positive'])
      weight_word = weight_word - float(SentiWordNet_df.loc[SentiWordNet_df['Word'] == word]['Negative'])
      #print(word,'=',weight_word)
  X_train_tfidf_df[n_gram] = weight_word * X_train_tfidf_df[n_gram]   
  X_test_tfidf_df[n_gram] = weight_word * X_test_tfidf_df[n_gram] 

# обучаем модель
svm = SGDClassifier()
svm.fit(X_train_tfidf_df, y_train)
predicted = svm.predict(X_test_tfidf_df)
print(metrics.classification_report(y_test, predicted))  

100%|██████████| 24309/24309 [59:24<00:00,  6.82it/s]


              precision    recall  f1-score   support

        20.0       0.31      0.47      0.37        17
        29.0       0.33      0.08      0.13        12
        40.0       0.62      0.33      0.43        24
        43.0       0.71      0.24      0.36        21
        57.0       0.18      0.57      0.27        28
        60.0       0.90      0.29      0.44        31
        65.0       0.21      0.10      0.14        29
        71.0       0.55      0.37      0.44        63
        80.0       0.65      0.52      0.58        61
        86.0       0.50      0.62      0.55       102
       100.0       0.60      0.64      0.62        83

    accuracy                           0.47       471
   macro avg       0.50      0.39      0.39       471
weighted avg       0.54      0.47      0.47       471



<a name="submit"></a>
# 5. Сохраняем submit

Лучший accuracy = 0,58 получился на векторизованных данных. Понимаю, что это очень низкое значение, но улучшить его не получилось. 

In [53]:
# преобразуем набор признаков в нормализованное представление tf-idf
tfidf = TfidfVectorizer(max_df=0.7, ngram_range=(1,5))
X_train_tfidf = tfidf.fit_transform(X_train['Title_Text_lemmatizer'])
X_test_tfidf = tfidf.transform(X_test['Title_Text_lemmatizer'])

# обучаем модель
svm = SGDClassifier()
svm.fit(X_train_tfidf, y_train)
predicted = svm.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted)) 

              precision    recall  f1-score   support

        20.0       0.43      0.35      0.39        17
        29.0       0.67      0.17      0.27        12
        40.0       1.00      0.33      0.50        24
        43.0       1.00      0.29      0.44        21
        57.0       0.43      0.32      0.37        28
        60.0       0.67      0.39      0.49        31
        65.0       0.88      0.24      0.38        29
        71.0       0.47      0.49      0.48        63
        80.0       0.77      0.56      0.65        61
        86.0       0.47      0.85      0.60       102
       100.0       0.69      0.80      0.74        83

    accuracy                           0.57       471
   macro avg       0.68      0.44      0.48       471
weighted avg       0.64      0.57      0.55       471



Поэтому векторизируем через TfidfVectorizer текст в test и сохраним результат в csv-файл.

In [55]:
data_test['Title'] = data_test['Review_Title'].apply(text_change)
data_test['Text'] = data_test['Review_Text'].apply(text_change)
data_test['Title_lemmatizer'] = data_test['Title'].apply(my_lemmatizer)
data_test['Text_lemmatizer'] = data_test['Text'].apply(my_lemmatizer)

data_test['Title_Text_lemmatizer'] = data_test['Title_lemmatizer']+' '+data_test['Text_lemmatizer']

# преобразуем набор признаков в нормализованное представление tf-idf
X_test_tfidf = tfidf.transform(data_test['Title_Text_lemmatizer'])

# сохраняем submit
submit = pd.DataFrame.from_dict({'Id':data_test.Id.values, 'Rating': svm.predict(X_test_tfidf)})
submit[['Id','Rating']].to_csv("sumbit.csv", index=False)
submit[['Id','Rating']].head()

,Id,Rating
0,2351,100.0
1,2352,100.0
2,2353,86.0
3,2354,60.0
4,2355,86.0
